In [ ]:
!earthengine authenticate

In [33]:
import ee
import os
from abd import abd

In [34]:
# Initialize Google Earth Engine
ee.Initialize()

# Data path
folderRoot = os.getcwd() + "/output"
if not os.path.exists(folderRoot):
    os.mkdir(folderRoot)

# Images path
folderCache = os.getcwd() + "/cache"
if not os.path.exists(folderCache):
    os.mkdir(folderCache)

In [ ]:
# fix permissions
!chmod 777 $folderRoot
!chmod 777 $folderCache

In [35]:
# create algorithm object
algorithm = abd.Abd(lat_lon="-83.50124371805877,41.88435023280987,-83.07548096199702,41.65275061592091",
                    date="2019-06-03",
                    days_threshold=180,
                    model="ocsvm",
                    sensor="modis",
                    cache_path=folderCache)


Selected sensor: Modis MOD09GA.006 [GEE Dataset = MODIS/006/MOD09GA]

Check if requested date is good for processing [2019-06-03]...
Extracting geometry (0) 1 of 1...
Pixel and cloud score for geometry #1: 1.0 and 0.73926!
Ok, image is good for processing: 1

Statistics: scale=500 meters, pixels=4845, date_start='2018-12-05', date_end='2019-11-30', interval_images='360', interval_unique_images='360', water_mask_images='16', days_threshold='180', morph_op='None', morph_op_iters='1', convolve='False', convolve_radius='1', scaler='robust', model='ocsvm', outliers_zscore='3.0', attributes='['cloud', 'ndvi', 'fai']', attribute_doy='False'


In [ ]:
# creating timeseries based on days_thresholds
algorithm.process_timeseries_data()


Starting time series processing ...
Trying to extract it from the cache...
Error trying to get it from cache: either doesn't exist or is corrupted! Creating it again...
Processing date [2018-12-05]...
Processing date [2018-12-06]...
Processing date [2018-12-07]...
   pixel index        date doy                 lat                 lon  \
0      0     0  2018-12-07   0  -83.07844326358365  41.657125512832515   
1      1     0  2018-12-07   0  -83.07844326358365   41.66161708925311   
2      2     0  2018-12-07   0  -83.07844326358365   41.66610866567371   
3      3     0  2018-12-07   0  -83.07844326358365    41.6706002420943   
4      4     0  2018-12-07   0  -83.07844326358365    41.6750918185149   

     cloud                 ndvi                    fai  
0  99999.0  -0.7838709677419355   -0.03759241206030151  
1  99999.0  -0.7970479704797049  -0.033823165829145733  
2  99999.0  -0.7746478873239436   -0.02959522613065326  
3  99999.0  -0.7407407407407408  -0.027824773869346738  
4  9

In [ ]:
# creating train and grid datasets
algorithm.process_training_data(df=algorithm.df_timeseries)

In [ ]:
# start training process
algorithm.train()

In [ ]:
# apply detection algorithm
algorithm.detect()

In [ ]:
# validate using ROI - example (https://code.earthengine.google.com/)
# algorithm.validate_using_roi(path='users/gee-user-name/erie', rois=['2019-06-03_modis_regular', '2019-06-03_modis_anomaly'], labels=[0, 1])

In [ ]:
# save geojson
algorithm.save_geojsons(folder=folder+"/geojson")

In [ ]:
# save results
algorithm.save_dataset(df=algorithm.df_results, path=folder+'/results.csv')

In [ ]:
# save collection images
algorithm.save_timeseries_plot(df=algorithm.df_timeseries, path=folder+'/timeseries.png')
if isinstance(algorithm.df_image, pd.DataFrame) and not algorithm.df_image.empty:
    algorithm.save_detection_plot(path=folder+'/detection.png')
    algorithm.save_image_png(image=algorithm.image_clip, date=dt.strptime(date, "%Y-%m-%d"), path=folder+"/"+date+".png")
    algorithm.save_image_tiff(image=algorithm.image_clip, date=dt.strptime(date, "%Y-%m-%d"), path=folder+"/"+date+".zip", folderName=args.name)

In [ ]:
# save collection images (tiff and png)
if args.save_collection:
    algorithm.save_collection_tiff(folder=folder+"/tiff", folderName="abd_erie.tiff")
    algorithm.save_collection_png(folder=folder+"/png")

In [ ]:
# save preprocessing results
if args.save_train:
    algorithm.save_dataset(df=algorithm.df_timeseries, path=folder+'/timeseries.csv')
    algorithm.save_dataset(df=algorithm.df_train, path=folder+'/df_train.csv')
    algorithm.save_dataset(df=algorithm.df_gridsearch, path=folder+'/df_gridsearch.csv')
    algorithm.save_dataset(df=algorithm.df_image, path=folder+'/df_image.csv')